In [ ]:
import heapq as hq
import copy
import time
#The manhattan distance measures the total number of horizontal and vertical moves required to align each tile in a puzzle with its correct position in the goal state.
def a_star_heuristic_manhattan_distance(current_state, target_state):
    total_manhattan_tiles_distance = 0
    for row in range(len(current_state)):
        for col in range(len(current_state[row])):
            tile_value = current_state[row][col]
         #calculate the manhattan distance for non zero tiles
            if tile_value != target_state[row][col] and tile_value != 0:
                target_tile_position = [0, 0]
                for row_index in range(len(target_state)):
                    if tile_value in target_state[row_index]:
                        target_tile_position[0] = row_index
                        target_tile_position[1] = target_state[row_index].index(tile_value)
                        break

                # manhattan distance between the current tile position and its target tile position
                distance = abs(row - target_tile_position[0]) + abs(col - target_tile_position[1] )
                total_manhattan_tiles_distance += distance

    return total_manhattan_tiles_distance

# The misplaced tiles hruristic counts the numbe of tiles that are not in their correct position when compared to the goal state
def a_star_heuristic_misplaced_tiles(current_state, target_state):
    total_misplaced_tiles_distance = 0

    for row_index, row in enumerate(current_state):
        for col_index, tile in enumerate(row):
            if tile == 0:
                continue

            if tile != target_state[row_index][col_index]:
                total_misplaced_tiles_distance += 1

    return total_misplaced_tiles_distance


def verify_states(next_state, parent_state):
    # print("compare", next_state, parent_state)
    # Check if the next state and parent state are of same dimensions
    if len(next_state) != len(parent_state) or len(next_state[0]) != len(parent_state[0]):
        return False
    for row in range(len(next_state)):
        for col in range(len(next_state[row])):
            if next_state[row][col] != parent_state[row][col]:
                return False
    return True

class Swathi_8_puzzle:
# attributes
   # parent(swthi_8_puzzel): Parent Node, matchlessif Root
   # state(list): State of the Node
   # path_cost(int): Overall cost of Start node to the this node
   # estimate_cost(int): Estimated cost from goal node
   # children(list): List of the child nodes.

    def __init__(self, parent=None, state=None, path_state_cost=0, estimate_state_cost=0):
            self.parent = parent
            self.path_state_cost = path_state_cost
            self.estimate_state_cost = estimate_state_cost
            self.state = state
            self.child_nodes = []

    def __lt__(self, another):
        return (self.path_state_cost + self.estimate_state_cost) < (another.path_state_cost + another.estimate_state_cost)

    def __eq__(self, another):
        return self.state == another.state

    def childAdd(self, node, expand_cost=1):
        #add the node as a child of the node, by setting path cost depending on the mov cost
        node.path_state_cost = self.path_state_cost + expand_cost
        node.parent = self
        self.child_nodes.append(node)

    def generate_children(self):
        indices = [(row, col) for row in range(len(self.state)) for col in range(len(self.state[row])) if self.state[row][col] == 0]
        row_idx, col_idx = indices[0] if indices else (None, None)
        len_state = len(self.state)
        return_state = []

        def move_left():
            # Move the tile left by interchanging  with the tile to the left of it
            current_board_position  = [list(row) for row in self.state]
            current_board_position [row_idx][col_idx], current_board_position [row_idx][col_idx - 1] = current_board_position [row_idx][col_idx - 1], current_board_position [row_idx][col_idx]
            if self.parent:
              if not verify_states(current_board_position , self.parent.state):
                return_state.append(current_board_position)
            else:
                return_state.append(current_board_position )


        def move_up():
            # Move the tile up by interchanging  with the tile over it
            current_board_position  = [list(row) for row in self.state]
            current_board_position [row_idx][col_idx], current_board_position [row_idx - 1][col_idx] = current_board_position [row_idx - 1][col_idx], current_board_position [row_idx][col_idx]
            if self.parent:
              if not verify_states(current_board_position , self.parent.state):
                return_state.append(current_board_position)
            else:
                return_state.append(current_board_position )

        def move_right():
            # Move the tile right by interchanging  with the tile to the right of it
            current_board_position  = [list(row) for row in self.state]
            current_board_position [row_idx][col_idx], current_board_position [row_idx][col_idx + 1] = current_board_position [row_idx][col_idx + 1], current_board_position [row_idx][col_idx]
            if self.parent:
              if not verify_states(current_board_position , self.parent.state):
                return_state.append(current_board_position)
            else:
                return_state.append(current_board_position )

        def move_down():
            # Move the tile down by interchanging  with the tile under it
            current_board_position  = [list(row) for row in self.state]
            current_board_position [row_idx][col_idx], current_board_position [row_idx + 1][col_idx] = current_board_position [row_idx + 1][col_idx], current_board_position [row_idx][col_idx]
            if self.parent:
             if not verify_states(current_board_position , self.parent.state):
                return_state.append(current_board_position)
            else:
                return_state.append(current_board_position )


        # Do the moves on the avilable empty tile location
        if col_idx != 0:
            move_left()

        if row_idx != 0:
            move_up()

        if col_idx != (len_state - 1):
            move_right()

        if row_idx != (len_state - 1):
            move_down()

        return return_state


def general_search(current_state, target_state, heuristic_id):
   #heuristic_id is a function that gives cost to go from initial state to target sate
   #initial_state is the state where start search
   #target_state is the state that is the goal

    root = Swathi_8_puzzle(state=current_state)

    # Initialize the priority queue.
    primacy_sequence = []
    hq.heappush(primacy_sequence, root)


    visited = []

    # Initialize counters.
    max_nodes_in_memory = 1
    nodes_expanded = 0
    count = 0
    # When the primacy sequence is not empty
    while primacy_sequence:

        if max_iteration_count and (count > max_iteration_count-1):
            print("Iteration maximum number has been reached!")
            break
        max_nodes_in_memory = max(len(primacy_sequence), max_nodes_in_memory)
        present_node = hq.heappop(primacy_sequence)
        #print("Popped node: ", present_node.state)
       # print("g(n): ", present_node.path_state_cost)
        #print("h(n): ", present_node.estimate_state_cost)
        #print()

        if verify_states(present_node.state, target_state):
            print("result obtained")
            return nodes_expanded, max_nodes_in_memory,present_node.path_state_cost

        else:
            visited.append(present_node)
            enlarge_sequence_list = [state for state in present_node.generate_children() if state]
            # print(present_node.state, present_node.generate_children())
            if enlarge_sequence_list == []:
                continue

        # Over the iterated states are expanded .
        for expanded_state in enlarge_sequence_list:
            set_upnode = Swathi_8_puzzle(state=expanded_state, path_state_cost=present_node.path_state_cost+1)

            if((primacy_sequence and set_upnode in primacy_sequence) or (visited and set_upnode in visited)):
                continue

            #Calculate cost function based on the selected heuristic.
            if heuristic_id == 3:
                set_upnode.estimate_state_cost = a_star_heuristic_manhattan_distance(set_upnode.state, target_state)
            elif heuristic_id == 2:
                set_upnode.estimate_state_cost = a_star_heuristic_misplaced_tiles(set_upnode.state, target_state)

            present_node.childAdd(node=set_upnode)
            hq.heappush(primacy_sequence, set_upnode)
        # print("length:",len(primacy_sequence))
        count+=1
        nodes_expanded += 1
    print(heuristic_id)
    print("Solution is not there !")
    return -1

alg_dict = {1:"Uniform Cost Search", 2:"A* Misplaced Tile Heuristic", 3:"A* Manhattan Distance Heuristic"}
# class Eight_puzzel_default_test_case:
#   def __init__ (self,elements,depth):
#       self.elements=elements
#       self.depth=depth

#   def show(self):
#       print("Sequence",self.elements,"Depth",self.depth)
# #Default test cases given by Dr. Keogh -(all solvable)
#
# Depth_list=[
#   Eight_puzzel_default_test_case([[1,2,3],[4,5,6],[7,8,0]],0),
#   Eight_puzzel_default_test_case([[1,2,3],[4,5,6],[0,7,8]],2),
#   Eight_puzzel_default_test_case([[1,2,3],[5,0,6],[4,7,8]],4),
#   Eight_puzzel_default_test_case([[1,3,6],[5,0,2],[4,7,8]],8),
#   Eight_puzzel_default_test_case([[1,3,6],[5,0,7],[4,8,2]],12),
#   Eight_puzzel_default_test_case([[1,6,7],[5,0,3],[4,8,2]],16),
#   Eight_puzzel_default_test_case([[7,1,2],[4,8,5],[6,3,0]],20),
#   Eight_puzzel_default_test_case([[0,7,2],[4,6,1],[3,5,8]],24)
# ]
# max_iteration_count = 0
# goal_state_matrix = [[1,2,3],[4,5,6],[7,8,0]]
# # result = general_search(Depth_list[0].elements, goal_state_matrix, 1)
# for dl in Depth_list:
#   print("Depth:",dl.depth)
#   print()
#   for alg_option in [1,2,3]:
#     start_time = time.time()
#     max_iteration_count = 0
#     result = general_search(dl.elements, goal_state_matrix, alg_option)
#     stop_time = time.time()
#     execution_time_ms = round((stop_time - start_time) * 1000, 2)

#     print("Algorithm:",alg_dict[alg_option])
#     print("Time:",execution_time_ms)
#     print("Max Queue Size: ", result[1])
#     print("Nodes Expanded:", result[0])
#     print(" ")




print("Welcome to Swathi's 8-puzzle solver\n\nType '1' to try the default puzzle\n\nTo make it more interesting,\nType '2' to create your own puzzle (preferred option)")
def get_user_choice():
    user_opt = int(input("ENTER OPTION VALUE\n"))
    while user_opt not in [1,2]:
        if user_opt not in [1,2]:
            print("Please enter either of the options 1 or 2")
            user_opt = int(input("ENTER CHOICE\n"))
    return user_opt
choice = get_user_choice()
if choice == 1:
    print("Thank you for your choice entry. In this option, we shall be using the default matrix for our 8-puzzle")
    #initial state matrix
    initial_state_matrix = [[1, 2, 0], [4, 5, 3], [7, 8, 6]]
    #goal state matrix
    goal_state_matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
elif choice == 2:
    dimension = 3
    initial_state_matrix = []
    goal_state_matrix = []
    print(f"Good choice, let's make our own 8-puzzle\nLet's start off by making the initial state {dimension}x{dimension} matrix followed by the goal state {dimension}x{dimension} matrix\n")
    print(f"Enter the elements of the initial state of the {dimension}x{dimension} puzzle, one row at a time. Enter 0 for an empty cell.")
    for i in range(0, dimension):
        invalidInput = False
        while True:
            row_i = list(map(int, input(f"\nEnter the row {i+1} elements : ").strip().split()))[:dimension]
            if len(row_i)<dimension:
                print("Invalid Input. You must enter",dimension,"elements")
            else:
                break
        initial_state_matrix.append(row_i)

    print("\nInitial state matrix\n", initial_state_matrix)
    print(f"\nEnter the elements of the goal state of the {dimension}x{dimension} puzzle, one row at a time. Enter 0 for an empty cell.")
    for i in range(0, dimension):
        row_i = list(map(int, input(f"\nEnter the row {i+1} elements : ").strip().split()))[:dimension]
        goal_state_matrix.append(row_i)

    print("Goal state matrix\n", goal_state_matrix)

print("Select choice of algorithm:\n")
print(" 1. Uniform Cost Search\n")
print(" 2. A* Misplaced Tile Heuristic\n")
print(" 3. A* Manhattan Distance Heuristic\n")


def get_alg_choice():
  while True:
    alg_choice = int(input("Enter your choice: "))
    if alg_choice in [1, 2, 3]:
      print(f"Selected Algorithm is ", alg_dict[alg_choice])
      return alg_choice
    else:
      print("Invalid choice. Please enter a valid choice.")
alg_option = int(get_alg_choice())

print("Enter the number of max iteration you want to run the Expand Function:\n")
max_iteration_count = int(input())

start_time = time.time()
# Initialize the algorithm
result = general_search(initial_state_matrix, goal_state_matrix, alg_option)
stop_time = time.time()
execution_time_ms = round((stop_time - start_time) * 1000, 2)
print("Success!")
print("The Initial state matrix has matched the desired Goal State Matrix!")
if(result==-1):
  print("solution doesnt exist")
else:
  print("Count of node_expanded:", result[0])
  print("Priority Queue node count:", result[1])
  print("Depth of the tree:", result[2])

print("-> %s seconds <-" % execution_time_ms)

Welcome to Swathi's 8-puzzle solver

Type '1' to try the default puzzle

To make it more interesting,
Type '2' to create your own puzzle (preferred option)
ENTER OPTION VALUE
2
Good choice, let's make our own 8-puzzle
Let's start off by making the initial state 3x3 matrix followed by the goal state 3x3 matrix

Enter the elements of the initial state of the 3x3 puzzle, one row at a time. Enter 0 for an empty cell.

Enter the row 1 elements : 1 2 3

Enter the row 2 elements : 4 5 6 

Enter the row 3 elements : 0 7 8 

Initial state matrix
 [[1, 2, 3], [4, 5, 6], [0, 7, 8]]

Enter the elements of the goal state of the 3x3 puzzle, one row at a time. Enter 0 for an empty cell.

Enter the row 1 elements : 1 2 3

Enter the row 2 elements : 4 5 6

Enter the row 3 elements : 7 8 0
Goal state matrix
 [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
Select choice of algorithm:

 1. Uniform Cost Search

 2. A* Misplaced Tile Heuristic

 3. A* Manhattan Distance Heuristic

Enter your choice: 2
Selected Algorithm 